# MICrONS Neuron Mesh Download
This tutorial walks through the key functions needed to access the MICrONS dataset programmatically and highlights key resources within it. While this tutorial is written for the MICrONS dataset specifically, the underlying technology (CAVE) is being used for multiple connectomics dataset. So, the interface presented here can be used to query them as well.

__This notebook is made to run in Google Colab. Each time your start the remote kernel, you will need to reinstall CAVE and add your token__

## CAVEclient and setup

The __CAVEclient__ is a python library that facilitates communication with a CAVE system. It can be install with

In [ ]:
!pip install -q caveclient 

and imported like so:

In [ ]:
import caveclient

## CAVE account setup

In order to manage server traffic, every user needs to create a CAVE account and download a user token to access CAVE's services programmatically. The CAVE infrastructure can be read about in more detail on our preprint. The MICrONS data is publicly available which means that no extra permissions need to be given to a new user account to access the data. Bulk downloads of some static data are also available without an account on MICrONs Explorer.

__A Google account (or Google-enabled account) is required to create a CAVE account.__

Go to: https://global.daf-apis.com/auth/api/v1/user/token to view a list of your existing tokens

If you have never made a token before:
1. go here: https://minnie.microns-daf.com/materialize/views/datastack/minnie65_public to accept terms of service
2. then go here https://global.daf-apis.com/auth/api/v1/create_token to create a new token.

### Set or save your token

From the website that just opened up, paste your token here:

In [ ]:
my_token = "PASTE_TOKEN_HERE"

## Libraries for this notebook

In [ ]:
from meshparty import trimesh_io, trimesh_vtk
import cloudvolume 
import numpy as np

In [ ]:
# Inititalize the CAVEclient
datastack_name = "minnie65_public"

client = caveclient.CAVEclient(datastack_name, auth_token=my_token)

# specify the materialization version, for consistency across time",
client.version = 943

__Versioning note:__ We use version 943 is this notebook because it corresponds with the most recent static segmentation, which is what you see on [microns-explorer.org](https://www.microns-explorer.org/gallery-mm3)

The most recent public version at the time of writing (2/1/2025) is __version 1300__.

## What is a Mesh?
A mesh is a set of vertices connected via triangular faces to form a 3 dimensional representation of the outer membrane of a neuron, glia or nucleus.

### Meshes can either be static or dynamic:
##### Static:
- pros: smaller files thus easier to work with, multiple levels of detail (lod) which can be accessed (example below)
- cons: may include false gaps and merges from self contacts, updated less frequently

##### Dynamic:
- pros: highly detailed thus more reflective of biological reality and backed by proofreading infrastructure CAVE (Connectome Annotation Versioning Engine)
- cons: much larger files, only one level of detail

In [ ]:
# to access dynamic meshes, you can query the segmentation source from the info client
client.info.segmentation_source()

In [ ]:
# this can be used to initialize a cloudvolume object
cv = cloudvolume.CloudVolume(client.info.segmentation_source(), progress=False, use_https=True)

Given a root_id, __CloudVolume__ can be used to retrieve the mesh. 
`cloudvolume.mesh.get()` returns a dictionary with the neuron segment id as the key and the mesh as the value

In [ ]:
# Example: pyramidal cell in v943
example_cell_id = 864691135572530981
%time mesh = cv.mesh.get(example_cell_id)[example_cell_id]

The returned mesh includes:
* vertices : Nx3 [x, y, z] positions in nm
* faces: Kx3 [i, j, k] indices into vertices that describe the triangular meshes

In [ ]:
print(mesh.vertices.shape, mesh.faces.shape)

Since downloading meshes can take some time, particularly for these dynamic meshes,
it is convient to cache them on disk.     
     
To facilitate the analysis of meshes, we developed a package called __MeshParty__ that we will use here to enable a cache.

In [ ]:
# to enable a cache, create a MeshMeta object
mm = trimesh_io.MeshMeta(cv_path = client.info.segmentation_source(),
                         disk_cache_path='minnie65_meshes',
                         map_gs_to_https=True)

You can get a mesh like this and it will be cached in memory and in disk in case you need it again.    
Restart the kernel and run the below cells again to see the difference.   

You'll find the mesh file saved as an hdf5 file in the "minnie65_meshes"
subdirectory

In [ ]:
mesh = mm.mesh(seg_id=example_cell_id)

The MeshParty object has more useful properties and attributes
such as:
* a scipy.csgraph sparse graph object (mesh.csgraph)
*  a networkx graph object (mesh.nxgraph) 

Read more about what you can do with MeshParty on its [Documentation](https://meshparty.readthedocs.io/en/latest/?badge=latest).

In particular it lets you associate _skeletons_, and _annotations_ onto the mesh into a "meshwork" object. 

## Static Meshes    
The meshes that are available in the visualization on microns-explorer.org are faster because they are static and have been downsampled multiple times. However, this comes with the drawback of being less biologically accurate.

Level of detail is controlled with optional argument, where `lod=0` is the highest level of detail

we can access one of these downsampled static meshes by setting the path here:

In [ ]:
## Previous versions of this tutorial have used v117 meshes, with the following path
# cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)

cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg_m943", use_https=True)

In [ ]:
# the cloud volume interface is the same but it is a faster initial download 
%time mesh = cv.mesh.get(example_cell_id, lod=3)[example_cell_id]

In [ ]:
# as you can see the meshes aren't exactly the same as before. They because they have not been downsampled
mesh.vertices.shape, mesh.faces.shape

In addition, the Static meshes are available in 3 levels of detail, this covers two orders of magnitude of detail
which is what neuroglancer leverages to efficiently load the data at the resolution necessary to render the current scene.  

In [ ]:
for lod in range(4):
    mesh = mesh = cv.mesh.get(example_cell_id, lod=lod)[example_cell_id]
    print(f"level of detail {lod}: n_verts: {mesh.vertices.shape[0]} n_faces: {mesh.faces.shape[0]}")

Now you have the neuron mesh, at your chosen level of detail, downloaded and ready to use locally. For next steps, consider:

1. Mesh repair and visualization [examples with __meshparty__](https://meshparty.readthedocs.io/en/latest/guide/visualization.html) 
2. Mesh simplification and analysis [from our friends at __Navis__](https://navis-org.github.io/navis/generated/gallery/4_remote/tutorial_remote_02_microns/)
3. Exporting the meshes into Neuron, or simulation program of your choice